# Course:  Convolutional Neural Networks for Image Classification

## Section-7
### What is next?
#### Deep CNN models with big filters

**Description:**  
*Train designed deep CNN models with big filters in the 1st convolutional layer  
Create video to visualize training process of the filters*  

**File:** *what_is_next_part_1.ipynb*

### Algorithm:

**--> Step 1:** Open preprocessed dataset  
**--> Step 2:** Load saved 1st deep CNN model  
**--> Step 3:** Train updated model on one preprocessed dataset  
**--> Step 4:** Predict with test dataset  
**--> Step 5:** Visualize filters of convolutional layer  
**--> Step 6:** Convert intermediate weights into images  


**Result:**  
- Images of intermediate weights

## Importing libraries

In [ ]:
# Importing needed libraries
import matplotlib.pyplot as plt
import numpy as np
import h5py


from keras.models import load_model, model_from_json
from keras.utils.np_utils import to_categorical
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback


## Setting up full paths

In [ ]:
# Full or absolute path to 'Section4' with preprocessed datasets
# (!) On Windows, the path should look like following:
# r'C:\Users\your_name\PycharmProjects\CNNCourse\Section4'
# or:
# 'C:\\Users\\your_name\\PycharmProjects\\CNNCourse\\Section4'
full_path_to_Section4 = \
    '/home/valentyn/PycharmProjects/CNNCourse/Section4'


# Full or absolute path to 'Section5' with designed models
# (!) On Windows, the path should look like following:
# r'C:\Users\your_name\PycharmProjects\CNNCourse\Section5'
# or:
# 'C:\\Users\\your_name\\PycharmProjects\\CNNCourse\\Section5'
full_path_to_Section5 = \
    '/home/valentyn/PycharmProjects/CNNCourse/Section5'


## Defining function to plot filters

In [ ]:
# Defining function to collect all filters in one grid
def convert_to_grid(x_input):
    # Getting shape values of the input
    number, height, width, channels = x_input.shape
    
    # Calculating grid size, number of rows and columns
    # Every element of the grid represents one filter
    grid_size = int(np.ceil(np.sqrt(number)))
    
    # Calculating height and width of entire grid according to
    # number of pixels along X and Y axes
    # Adding extra pixels to be as gaps in between filters
    grid_height = height * grid_size + (grid_size - 1)
    grid_width = width * grid_size + (grid_size - 1)
    
    # Creating Numpy array for the grid
    # Filling all grid pixels with white colour
    grid = np.zeros((grid_height, grid_width, channels)) + 255
    
    # Defining index for current filter
    n = 0
    
    # Defining positions for current filter along Y axis of the grid
    y_min, y_max = 0, height
    
    # Iterating all grid elements and filling them with filters' values
    # Iterating every row of the grid elements
    for y in range(grid_size):
        # Defining positions for current filter along X axis of the grid
        x_min, x_max = 0, width
        
        # Iterating every column of the current row
        for x in range(grid_size):
            # Checking if current index of the filter is less 
            # than total number of filters
            if n < number:
                # Getting current filter from the input
                filter_current = x_input[n]
                
                # Extracting minimum and maximum values of current filter
                f_min, f_max = np.min(filter_current), np.max(filter_current)
                
                # Scaling filter's values to be in range [0, 255]
                # Assigning scaled values to appropriate positions in the grid
                grid[y_min:y_max, x_min:x_max] = 255.0 * (filter_current - f_min) / (f_max - f_min)
                
                # Increasing index of current filter
                n += 1
            
            # Moving positions for current filter along X axis of the grid
            # Adding 1 pixel to be as gap in between filters
            x_min += width + 1
            x_max += width + 1
        
        # Moving positions for current filter along Y axis of the grid
        # Adding 1 pixel to be as gap in between filters
        y_min += height + 1
        y_max += height + 1
        
    # Returning grid filled with filters
    return grid


# Check point
print('Function to create a grid is successfully defined')
print()


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved custom dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 
               'custom' + '/' + 
               'dataset_custom_rgb_255_mean.hdf5', 'r') as f:
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[5])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 5)
y_validation = to_categorical(y_validation, num_classes = 5)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print(y_test.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[5])


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 2: Loading saved 1st model

In [ ]:
# Loading 1st model for custom dataset
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb = load_model(full_path_to_Section5 + '/' + 'custom' + '/' + 'model_1_custom_rgb.h5')


# Check point
print('Model is successfully loaded')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


In [ ]:
# Assigning new spatial dimension for the filters of the 1st convolutional layer
model_rgb.layers[0].kernel_size = (32, 32)


# Re-loading model and initializing it by new structure
model_rgb = model_from_json(model_rgb.to_json())


# Re-compiling model
model_rgb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Check point
print('Model is successfully re-loaded')
print('Model is successfully re-compiled')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 3: Training

In [ ]:
# Defining number of epochs
epochs = 50


# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** (x + epochs), verbose=1)


# Check point
print('Number of epochs and schedule for learning rate are set successfully')


In [ ]:
# Preparing filepath to save best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
best_weights_filepath = 'custom' + '/' + 'w_1_custom_rgb_255_mean.h5'


# Defining schedule to save best weights
best_weights = ModelCheckpoint(filepath=best_weights_filepath,
                               save_weights_only=True,                                   
                               monitor='val_accuracy',
                               mode='max',
                               save_best_only=True,
                               period=1,
                               verbose=1)


# Check point
print('Schedule to save best weights is created')


In [ ]:
# Defining schedule to save intermediate weights
class CustomCallback(Callback):
    # Constructor of the class
    def __init__(self):
        # Defining variable to be as a part of filename
        self.filename = 0
        
    # Function that is called at the end of every batch
    def on_train_batch_end(self, batch, logs=None):
        # Checking if it is every 10th batch
        if batch % 10 == 0:
            # Preparing filepath to save intermediate weights
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            intermediate_weights_filepath = 'custom' + '/' + \
                                            'intermediate' + '/' + \
                                            '{0:04d}'.format(self.filename) + \
                                            '_w_1_custom_rgb_255_mean.h5'
            
            # Getting weights only for the first convolutional layer
            weights_layer_0 = self.model.get_weights()[0]
                
            # Saving obtained weights into new HDF5 binary file
            # Initiating File object
            # Creating file with current name
            # Opening it in writing mode by 'w'
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            with h5py.File(intermediate_weights_filepath, 'w') as f:
                # Calling method to create dataset of given shape and type
                # Saving Numpy array with weights from the first layer
                f.create_dataset('weights_layer_0', data=weights_layer_0, dtype='f')
                    
            # Increasing variable to be as a part of the next filename
            self.filename += 1


# Check point
print('Schedule to save intermediate weights is created')


In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Training model
h = model_rgb.fit(x_train, y_train,
                  batch_size=50,
                  epochs=epochs,
                  validation_data=(x_validation, y_validation),
                  callbacks=[learning_rate, best_weights, CustomCallback()],
                  verbose=1)


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 4: Predicting with test dataset

In [ ]:
# loading and assigning best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb.load_weights('custom' + '/' + 'w_1_custom_rgb_255_mean.h5')


# Check point
print('Best weights for 1st RGB model are loaded and assigned')


In [ ]:
# Testing RGB model
temp = model_rgb.predict(x_test)
        

# Check point
# Showing prediction shape and scores
print('prediction shape  :', temp.shape)  # (278, 5)
print('prediction scores :', temp[0])  # 5 score numbers
print()

    
# Getting indexes of maximum values along specified axis
temp = np.argmax(temp, axis=1)
        
        
# Check point
# Showing prediction shape after convertion
# Showing predicted and correct indexes of classes
print('prediction shape  :', temp.shape)  # (278,)
print('predicted indexes :', temp[0:10])
print('correct indexes   :', y_test[:10])
print()


# Calculating accuracy
# We compare predicted class with correct class for all input images
# By saying 'temp == y_test' we create Numpy array with True and False values
# By function 'np.mean' we calculate mean value:
# all_True / (all_True + all_False)
accuracy = np.mean(temp == y_test)


# Check point
# Showing calculated accuracy
print('Testing accuracy  : {0:.5f}'.format(accuracy))


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 5: Visualizing filters of convolutional layer

In [ ]:
# Getting all the weights from entire model
w = model_rgb.get_weights()


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


# Moving last dimension to the first position
# Reshaping Numpy array from
# (height, width, channels, number) to
# (number, height, width, channels)
w[0] = w[0].transpose(3, 0, 1, 2)


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Converting weights of convolutional layer into a grid
grid = convert_to_grid(w[0])


# Plotting the grid
plt.imshow(grid.astype('uint8'))


# Switching off axes
# Setting size of the plot (width, height) in inches
plt.axis('off')
plt.gcf().set_size_inches(10, 10)


# Showing the plot
plt.show()


### Custom, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 6: Converting intermediate weights into images

In [ ]:
# Iterating all saved intermediate weights
for i in range(350):
    # Opening saved binary file with intermediate weights
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    with h5py.File('custom/intermediate/' + '{0:04d}'.format(i) + \
                   '_w_1_custom_rgb_255_mean.h5', 'r') as f:
        # Extracting saved array with intermediate weights by appropriate key
        # Saving them into new variable
        weights = f['weights_layer_0']  # HDF5 dataset

        # Converting them into Numpy array
        weights = np.array(weights)  # Numpy arrays

    
    # Moving last dimension to the first position
    # Reshaping Numpy array from
    # (height, width, channels, number) to
    # (number, height, width, channels)
    weights = weights.transpose(3, 0, 1, 2)
    
    
    # Converting weights of convolutional layer into a grid
    grid = convert_to_grid(weights)
    
    
    # Plotting the grid
    plt.imshow(grid.astype('uint8'))
    
    
    # Switching off axes
    # Setting size of the plot (width, height) in inches
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10)
    
    
    # Saving the plot into image file
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    plt.savefig('custom/intermediate/images/' + '{0:04d}'.format(i) + '.png', dpi=300)
    
    
    # Closing the plot
    plt.close()
    
    
    # Check point
    print('{0:04d}.png is created'.format(i))


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved CIFAR-10 dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 
               'cifar10' + '/' + 
               'dataset_cifar10_rgb_255_mean.hdf5', 'r') as f:
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[5])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 10)
y_validation = to_categorical(y_validation, num_classes = 10)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print(y_test.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[5])


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 2: Loading saved 1st model

In [ ]:
# Loading 1st model for CIFAR-10 dataset
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb = load_model(full_path_to_Section5 + '/' + 'cifar10' + '/' + 'model_1_cifar10_rgb.h5')


# Check point
print('Model is successfully loaded')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


In [ ]:
# Assigning new spatial dimension for the filters of the 1st convolutional layer
model_rgb.layers[0].kernel_size = (16, 16)


# Re-loading model and initializing it by new structure
model_rgb = model_from_json(model_rgb.to_json())


# Re-compiling model
model_rgb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Check point
print('Model is successfully re-loaded')
print('Model is successfully re-compiled')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 3: Training

In [ ]:
# Defining number of epochs
epochs = 50


# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** (x + epochs), verbose=1)


# Check point
print('Number of epochs and schedule for learning rate are set successfully')


In [ ]:
# Preparing filepath to save best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
best_weights_filepath = 'cifar10' + '/' + 'w_1_cifar10_rgb_255_mean.h5'


# Defining schedule to save best weights
best_weights = ModelCheckpoint(filepath=best_weights_filepath,
                               save_weights_only=True,
                               monitor='val_accuracy',
                               mode='max',
                               save_best_only=True,
                               period=1,
                               verbose=1)


# Check point
print('Schedule to save best weights is created')


In [ ]:
# Defining schedule to save intermediate weights
class CustomCallback(Callback):
    # Constructor of the class
    def __init__(self):
        # Defining variable to be as a part of filename
        self.filename = 0
        
    # Function that is called at the end of every batch
    def on_train_batch_end(self, batch, logs=None):
        # Checking if it is every 100th batch
        if batch % 100 == 0:
            # Preparing filepath to save intermediate weights
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            intermediate_weights_filepath = 'cifar10' + '/' + \
                                            'intermediate' + '/' + \
                                            '{0:04d}'.format(self.filename) + \
                                            '_w_1_cifar10_rgb_255_mean.h5'
            
            # Getting weights only for the first convolutional layer
            weights_layer_0 = self.model.get_weights()[0]
                
            # Saving obtained weights into new HDF5 binary file
            # Initiating File object
            # Creating file with current name
            # Opening it in writing mode by 'w'
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            with h5py.File(intermediate_weights_filepath, 'w') as f:
                # Calling method to create dataset of given shape and type
                # Saving Numpy array with weights from the first layer
                f.create_dataset('weights_layer_0', data=weights_layer_0, dtype='f')
                    
            # Increasing variable to be as a part of the next filename
            self.filename += 1


# Check point
print('Schedule to save intermediate weights is created')


In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Training model
h = model_rgb.fit(x_train, y_train,
                  batch_size=50,
                  epochs=epochs,
                  validation_data=(x_validation, y_validation),
                  callbacks=[learning_rate, best_weights, CustomCallback()],
                  verbose=1)


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 4: Predicting with test dataset

In [ ]:
# loading and assigning best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb.load_weights('cifar10' + '/' + 'w_1_cifar10_rgb_255_mean.h5')


# Check point
print('Best weights for 1st RGB model are loaded and assigned')


In [ ]:
# Testing RGB model
temp = model_rgb.predict(x_test)
        

# Check point
# Showing prediction shape and scores
print('prediction shape  :', temp.shape)  # (10000, 10)
print('prediction scores :', temp[0, 0:5])  # 5 score numbers
print()

    
# Getting indexes of maximum values along specified axis
temp = np.argmax(temp, axis=1)
        
        
# Check point
# Showing prediction shape after convertion
# Showing predicted and correct indexes of classes
print('prediction shape  :', temp.shape)  # (10000,)
print('predicted indexes :', temp[0:10])
print('correct indexes   :', y_test[:10])
print()


# Calculating accuracy
# We compare predicted class with correct class for all input images
# By saying 'temp == y_test' we create Numpy array with True and False values
# By function 'np.mean' we calculate mean value:
# all_True / (all_True + all_False)
accuracy = np.mean(temp == y_test)


# Check point
# Showing calculated accuracy
print('Testing accuracy  : {0:.5f}'.format(accuracy))


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 5: Visualizing filters of convolutional layer

In [ ]:
# Getting all the weights from entire model
w = model_rgb.get_weights()


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


# Moving last dimension to the first position
# Reshaping Numpy array from
# (height, width, channels, number) to
# (number, height, width, channels)
w[0] = w[0].transpose(3, 0, 1, 2)


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Converting weights of convolutional layer into a grid
grid = convert_to_grid(w[0])


# Plotting the grid
plt.imshow(grid.astype('uint8'))


# Switching off axes
# Setting size of the plot (width, height) in inches
plt.axis('off')
plt.gcf().set_size_inches(10, 10)


# Showing the plot
plt.show()


### CIFAR-10, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 6: Converting intermediate weights into images

In [ ]:
# Iterating all saved intermediate weights
for i in range(450):
    # Opening saved binary file with intermediate weights
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    with h5py.File('cifar10/intermediate/' + '{0:04d}'.format(i) + \
                   '_w_1_cifar10_rgb_255_mean.h5', 'r') as f:
        # Extracting saved array with intermediate weights by appropriate key
        # Saving them into new variable
        weights = f['weights_layer_0']  # HDF5 dataset

        # Converting them into Numpy array
        weights = np.array(weights)  # Numpy arrays

    
    # Moving last dimension to the first position
    # Reshaping Numpy array from
    # (height, width, channels, number) to
    # (number, height, width, channels)
    weights = weights.transpose(3, 0, 1, 2)
    
    
    # Converting weights of convolutional layer into a grid
    grid = convert_to_grid(weights)
    
    
    # Plotting the grid
    plt.imshow(grid.astype('uint8'))
    
    
    # Switching off axes
    # Setting size of the plot (width, height) in inches
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10)
    
    
    # Saving the plot into image file
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    plt.savefig('cifar10/intermediate/images/' + '{0:04d}'.format(i) + '.png', dpi=300)
    
    
    # Closing the plot
    plt.close()
    
    
    # Check point
    print('{0:04d}.png is created'.format(i))


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved MNIST dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 
               'mnist' + '/' + 
               'dataset_mnist_gray_255_mean.hdf5', 'r') as f:
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables    
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[5])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 10)
y_validation = to_categorical(y_validation, num_classes = 10)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print(y_test.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[5])


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 2: Loading saved 1st model

In [ ]:
# Loading 1st model for MNIST dataset
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_gray = load_model(full_path_to_Section5 + '/' + 'mnist' + '/' + 'model_1_mnist_gray.h5')


# Check point
print('Model is successfully loaded')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_gray.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_gray.get_weights()
print(w[0].shape)


In [ ]:
# Assigning new spatial dimension for the filters of the 1st convolutional layer
model_gray.layers[0].kernel_size = (14, 14)


# Re-loading model and initializing it by new structure
model_gray = model_from_json(model_gray.to_json())


# Re-compiling model
model_gray.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Check point
print('Model is successfully re-loaded')
print('Model is successfully re-compiled')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_gray.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_gray.get_weights()
print(w[0].shape)


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 3: Training

In [ ]:
# Defining number of epochs
epochs = 50


# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** (x + epochs), verbose=1)


# Check point
print('Number of epochs and schedule for learning rate are set successfully')


In [ ]:
# Preparing filepath to save best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
best_weights_filepath = 'mnist' + '/' + 'w_1_mnist_gray_255_mean.h5'


# Defining schedule to save best weights
best_weights = ModelCheckpoint(filepath=best_weights_filepath,
                               save_weights_only=True,                                   
                               monitor='val_accuracy',
                               mode='max',
                               save_best_only=True,
                               period=1,
                               verbose=1)


# Check point
print('Schedule to save best weights is created')


In [ ]:
# Defining schedule to save intermediate weights
class CustomCallback(Callback):
    # Constructor of the class
    def __init__(self):
        # Defining variable to be as a part of filename
        self.filename = 0
        
    # Function that is called at the end of every batch
    def on_train_batch_end(self, batch, logs=None):
        # Checking if it is every 100th batch
        if batch % 100 == 0:
            # Preparing filepath to save intermediate weights
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            intermediate_weights_filepath = 'mnist' + '/' + \
                                            'intermediate' + '/' + \
                                            '{0:04d}'.format(self.filename) + \
                                            '_w_1_mnist_gray_255_mean.h5'
            
            # Getting weights only for the first convolutional layer
            weights_layer_0 = self.model.get_weights()[0]
                
            # Saving obtained weights into new HDF5 binary file
            # Initiating File object
            # Creating file with current name
            # Opening it in writing mode by 'w'
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            with h5py.File(intermediate_weights_filepath, 'w') as f:
                # Calling method to create dataset of given shape and type
                # Saving Numpy array with weights from the first layer
                f.create_dataset('weights_layer_0', data=weights_layer_0, dtype='f')
                    
            # Increasing variable to be as a part of the next filename
            self.filename += 1


# Check point
print('Schedule to save intermediate weights is created')


In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Training model
h = model_gray.fit(x_train, y_train,
                   batch_size=50,
                   epochs=epochs,
                   validation_data=(x_validation, y_validation),
                   callbacks=[learning_rate, best_weights, CustomCallback()],
                   verbose=1)


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 4: Predicting with test dataset

In [ ]:
# loading and assigning best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_gray.load_weights('mnist' + '/' + 'w_1_mnist_gray_255_mean.h5')


# Check point
print('Best weights for 1st GRAY model are loaded and assigned')


In [ ]:
# Testing GRAY model
temp = model_gray.predict(x_test)


# Check point
# Showing prediction shape and scores
print('prediction shape  :', temp.shape)  # (10000, 10)
print('prediction scores :', temp[0, 0:5])  # 5 score numbers
print()

    
# Getting indexes of maximum values along specified axis
temp = np.argmax(temp, axis=1)
        
        
# Check point
# Showing prediction shape after convertion
# Showing predicted and correct indexes of classes
print('prediction shape  :', temp.shape)  # (10000,)
print('predicted indexes :', temp[0:10])
print('correct indexes   :', y_test[:10])
print()


# Calculating accuracy
# We compare predicted class with correct class for all input images
# By saying 'temp == y_test' we create Numpy array with True and False values
# By function 'np.mean' we calculate mean value:
# all_True / (all_True + all_False)
accuracy = np.mean(temp == y_test)


# Check point
# Showing calculated accuracy
print('Testing accuracy  : {0:.5f}'.format(accuracy))


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 5: Visualizing filters of convolutional layer

In [ ]:
# Getting all the weights from entire model
w = model_gray.get_weights()


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


# Moving last dimension to the first position
# Reshaping Numpy array from
# (height, width, channels, number) to
# (number, height, width, channels)
w[0] = w[0].transpose(3, 0, 1, 2)


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Converting weights of convolutional layer into a grid
grid = convert_to_grid(w[0])


# Plotting the grid
plt.imshow(grid.astype('uint8'), cmap='gray')


# Switching off axes
# Setting size of the plot (width, height) in inches
plt.axis('off')
plt.gcf().set_size_inches(10, 10)


# Showing the plot
plt.show()


### MNIST, 1st GRAY model
### GRAY dataset (255.0 ==> mean)

## Step 4: Converting intermediate weights into images

In [ ]:
# Iterating all saved intermediate weights
for i in range(550):
    # Opening saved binary file with intermediate weights
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    with h5py.File('mnist/intermediate/' + '{0:04d}'.format(i) + \
                   '_w_1_mnist_gray_255_mean.h5', 'r') as f:
        # Extracting saved array with intermediate weights by appropriate key
        # Saving them into new variable
        weights = f['weights_layer_0']  # HDF5 dataset

        # Converting them into Numpy array
        weights = np.array(weights)  # Numpy arrays

    
    # Moving last dimension to the first position
    # Reshaping Numpy array from
    # (height, width, channels, number) to
    # (number, height, width, channels)
    weights = weights.transpose(3, 0, 1, 2)
    
    
    # Converting weights of convolutional layer into a grid
    grid = convert_to_grid(weights)
    
    
    # Plotting the grid
    plt.imshow(grid.astype('uint8'), cmap='gray')
    
    
    # Switching off axes
    # Setting size of the plot (width, height) in inches
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10)
    
    
    # Saving the plot into image file
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    plt.savefig('mnist/intermediate/images/' + '{0:04d}'.format(i) + '.png', dpi=300)
    
    
    # Closing the plot
    plt.close()
    
    
    # Check point
    print('{0:04d}.png is created'.format(i))


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 1: Opening preprocessed dataset

In [ ]:
# Opening saved Traffic Signs dataset from HDF5 binary file
# Initiating File object
# Opening file in reading mode by 'r'
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
with h5py.File(full_path_to_Section4 + '/' + 
               'ts' + '/' + 
               'dataset_ts_rgb_255_mean.hdf5', 'r') as f:
    # Showing all keys in the HDF5 binary file
    print(list(f.keys()))
    
    # Extracting saved arrays for training by appropriate keys
    # Saving them into new variables    
    x_train = f['x_train']  # HDF5 dataset
    y_train = f['y_train']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_train = np.array(x_train)  # Numpy arrays
    y_train = np.array(y_train)  # Numpy arrays
    
    
    # Extracting saved arrays for validation by appropriate keys
    # Saving them into new variables 
    x_validation = f['x_validation']  # HDF5 dataset
    y_validation = f['y_validation']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_validation = np.array(x_validation)  # Numpy arrays
    y_validation = np.array(y_validation)  # Numpy arrays
    
    
    # Extracting saved arrays for testing by appropriate keys
    # Saving them into new variables 
    x_test = f['x_test']  # HDF5 dataset
    y_test = f['y_test']  # HDF5 dataset
    # Converting them into Numpy arrays
    x_test = np.array(x_test)  # Numpy arrays
    y_test = np.array(y_test)  # Numpy arrays


In [ ]:
# Showing types of loaded arrays
print(type(x_train))
print(type(y_train))
print(type(x_validation))
print(type(y_validation))
print(type(x_test))
print(type(y_test))
print()


# Showing shapes of loaded arrays
print(x_train.shape)
print(y_train.shape)
print(x_validation.shape)
print(y_validation.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
# Showing class index from the vector
print('Class index from vector:', y_train[5])
print()

# Preparing classes to be passed into the model
# Transforming them from vectors to binary matrices
# It is needed to set relationship between classes to be understood by the algorithm
# Such format is commonly used in training and predicting
y_train = to_categorical(y_train, num_classes = 43)
y_validation = to_categorical(y_validation, num_classes = 43)


# Showing shapes of converted vectors into matrices
print(y_train.shape)
print(y_validation.shape)
print(y_test.shape)
print()


# Showing class index from the matrix
print('Class index from matrix:', y_train[5])


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 2: Loading saved 1st model

In [ ]:
# Loading 1st model for Traffic Signs dataset
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb = load_model(full_path_to_Section5 + '/' + 'ts' + '/' + 'model_1_ts_rgb.h5')


# Check point
print('Model is successfully loaded')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


In [ ]:
# Assigning new spatial dimension for the filters of the 1st convolutional layer
model_rgb.layers[0].kernel_size = (24, 24)


# Re-loading model and initializing it by new structure
model_rgb = model_from_json(model_rgb.to_json())


# Re-compiling model
model_rgb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Check point
print('Model is successfully re-loaded')
print('Model is successfully re-compiled')


In [ ]:
# Check point
# Showing spatial size of filters for the 1st convolutional layer
print(model_rgb.layers[0].kernel_size)


# Check point
# Showing shapes of the weights for the 1st convolutional layer
w = model_rgb.get_weights()
print(w[0].shape)


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 3: Training

In [ ]:
# Defining number of epochs
epochs = 50


# Defining schedule to update learning rate
learning_rate = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** (x + epochs), verbose=1)


# Check point
print('Number of epochs and schedule for learning rate are set successfully')


In [ ]:
# Preparing filepath to save best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
best_weights_filepath = 'ts' + '/' + 'w_1_ts_rgb_255_mean.h5'


# Defining schedule to save best weights
best_weights = ModelCheckpoint(filepath=best_weights_filepath,
                               save_weights_only=True,                                   
                               monitor='val_accuracy',
                               mode='max',
                               save_best_only=True,
                               period=1,
                               verbose=1)


# Check point
print('Schedule to save best weights is created')


In [ ]:
# Defining schedule to save intermediate weights
class CustomCallback(Callback):
    # Constructor of the class
    def __init__(self):
        # Defining variable to be as a part of filename
        self.filename = 0
        
    # Function that is called at the end of every batch
    def on_train_batch_end(self, batch, logs=None):
        # Checking if it is every 100th batch
        if batch % 100 == 0:
            # Preparing filepath to save intermediate weights
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            intermediate_weights_filepath = 'ts' + '/' + \
                                            'intermediate' + '/' + \
                                            '{0:04d}'.format(self.filename) + \
                                            '_w_1_ts_rgb_255_mean.h5'
            
            # Getting weights only for the first convolutional layer
            weights_layer_0 = self.model.get_weights()[0]
                
            # Saving obtained weights into new HDF5 binary file
            # Initiating File object
            # Creating file with current name
            # Opening it in writing mode by 'w'
            # (!) On Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            with h5py.File(intermediate_weights_filepath, 'w') as f:
                # Calling method to create dataset of given shape and type
                # Saving Numpy array with weights from the first layer
                f.create_dataset('weights_layer_0', data=weights_layer_0, dtype='f')
                    
            # Increasing variable to be as a part of the next filename
            self.filename += 1


# Check point
print('Schedule to save intermediate weights is created')


In [ ]:
# If you're using Nvidia GPU and 'cnngpu' environment, there might be an issue like:
'''Failed to get convolution algorithm. This is probably because cuDNN failed to initialize'''
# In this case, close all Jupyter Notebooks, close Terminal Window or Anaconda Prompt
# Open again just this one Jupyter Notebook and run it


# Training model
h = model_rgb.fit(x_train, y_train,
                  batch_size=50,
                  epochs=epochs,
                  validation_data=(x_validation, y_validation),
                  callbacks=[learning_rate, best_weights, CustomCallback()],
                  verbose=1)


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 4: Predicting with test dataset

In [ ]:
# loading and assigning best weights
# (!) On Windows, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
model_rgb.load_weights('ts' + '/' + 'w_1_ts_rgb_255_mean.h5')


# Check point
print('Best weights for 1st RGB model are loaded and assigned')


In [ ]:
# Testing RGB model
temp = model_rgb.predict(x_test)


# Check point
# Showing prediction shape and scores
print('prediction shape  :', temp.shape)  # (3111, 43)
print('prediction scores :', temp[0, 0:5])  # 5 score numbers
print()

    
# Getting indexes of maximum values along specified axis
temp = np.argmax(temp, axis=1)
        
        
# Check point
# Showing prediction shape after convertion
# Showing predicted and correct indexes of classes
print('prediction shape  :', temp.shape)  # (3111,)
print('predicted indexes :', temp[0:10])
print('correct indexes   :', y_test[:10])
print()


# Calculating accuracy
# We compare predicted class with correct class for all input images
# By saying 'temp == y_test' we create Numpy array with True and False values
# By function 'np.mean' we calculate mean value:
# all_True / (all_True + all_False)
accuracy = np.mean(temp == y_test)


# Check point
# Showing calculated accuracy
print('Testing accuracy  : {0:.5f}'.format(accuracy))


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 5: Visualizing filters of convolutional layer

In [ ]:
# Getting all the weights from entire model
w = model_rgb.get_weights()


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


# Moving last dimension to the first position
# Reshaping Numpy array from
# (height, width, channels, number) to
# (number, height, width, channels)
w[0] = w[0].transpose(3, 0, 1, 2)


# Check point
# Showing shape of the weights for convolutional layer
print(w[0].shape)


In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Converting weights of convolutional layer into a grid
grid = convert_to_grid(w[0])


# Plotting the grid
plt.imshow(grid.astype('uint8'))


# Switching off axes
# Setting size of the plot (width, height) in inches
plt.axis('off')
plt.gcf().set_size_inches(10, 10)


# Showing the plot
plt.show()


### Traffic Signs, 1st RGB model
### RGB dataset (255.0 ==> mean)

## Step 6: Converting intermediate weights into images

In [ ]:
# Iterating all saved intermediate weights
for i in range(400):
    # Opening saved binary file with intermediate weights
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    with h5py.File('ts/intermediate/' + '{0:04d}'.format(i) + \
                   '_w_1_ts_rgb_255_mean.h5', 'r') as f:
        # Extracting saved array with intermediate weights by appropriate key
        # Saving them into new variable
        weights = f['weights_layer_0']  # HDF5 dataset

        # Converting them into Numpy array
        weights = np.array(weights)  # Numpy arrays

    
    # Moving last dimension to the first position
    # Reshaping Numpy array from
    # (height, width, channels, number) to
    # (number, height, width, channels)
    weights = weights.transpose(3, 0, 1, 2)
    
    
    # Converting weights of convolutional layer into a grid
    grid = convert_to_grid(weights)
    
    
    # Plotting the grid
    plt.imshow(grid.astype('uint8'))
    
    
    # Switching off axes
    # Setting size of the plot (width, height) in inches
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10)
    
    
    # Saving the plot into image file
    # (!) On Windows, it might need to change
    # this: + '/' +
    # to this: + '\' +
    # or to this: + '\\' +
    plt.savefig('ts/intermediate/images/' + '{0:04d}'.format(i) + '.png', dpi=300)
    
    
    # Closing the plot
    plt.close()
    
    
    # Check point
    print('{0:04d}.png is created'.format(i))


### Some comments

To get more details for usage of 'model_from_json':  
**print(help(model_from_json))**
  
More details and examples are here:  
 - https://keras.io/api/models/model_saving_apis/#modelfromjson-function  
  
  
To get more details for usage of 'to_json':  
**print(help(Sequential().to_json))**
  
More details and examples are here:  
 - https://keras.io/api/models/model_saving_apis/#tojson-method  


In [ ]:
print(help(model_from_json))

In [ ]:
from keras.models import Sequential

print(help(Sequential().to_json))